In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
import nltk

from tqdm import trange
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize

# For traditional ML pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [2]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Error loading punkt_tab: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [3]:
df = pd.read_csv('data/train.csv',engine='python', on_bad_lines= 'skip')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20822, 5)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20822 entries, 0 to 20821
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20822 non-null  object
 1   title   20257 non-null  object
 2   author  18847 non-null  object
 3   text    20763 non-null  object
 4   label   20800 non-null  object
dtypes: object(5)
memory usage: 813.5+ KB


In [6]:
df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [7]:
df.duplicated().sum()

0

In [8]:
train = df.copy()
train.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [9]:
# check for distinct values

for col in train.columns:
    distinct_vals = len(train[col].unique())
    print(f"{col}: {distinct_vals} unique values")

id: 20822 unique values
title: 19819 unique values
author: 4207 unique values
text: 20390 unique values
label: 5 unique values


In [10]:
train.drop(['id', 'title', 'author'], axis=1, inplace=True)
train.head(3)

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1


In [11]:
train.isna().sum()

text     59
label    22
dtype: int64

In [12]:
train.dropna(inplace=True)

In [13]:
train.isna().sum()

text     0
label    0
dtype: int64

In [14]:
train['label'].value_counts()

label
0                                                   10387
1                                                   10372
 как люди воспринимают своё положение.                  1
 чтобы это была дорога с двусторонним движением.        1
Name: count, dtype: int64

In [17]:
train = train[train['label'].isin(['0', '1'])]
train.shape

(20759, 2)